# Preparing the dataset
- Read in the scraped data
- Add the ratings from IMDB
- Add the subtitles as text

In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
movies = pd.read_csv('f_csv/movie_1.csv')
movies

,title,description
0,1. Margin Call (2011),Set in the early stages of the 2008 financial ...
1,2. Wall Street (1987),This is the classic film that started it all. ...
2,3. The Big Short (2015),"Based on the book by Michael Lewis, The Big Sh..."
3,4. Trading Places (1983),"Being the only pure comedy on this list, Tradi..."
4,5. The Wolf of Wall Street (2013),"Directed by Martin Scorsese, The Wolf of Wall ..."
5,6. American Psycho (2000),American Psycho is a cult classic in Wall Stre...
6,7. Arbitrage (2012),Arbitrage is a 2012 film that tells the story ...
7,8. Equity (2016),Equity follows the story of Naomi Bishop (Anna...
8,9. Inside Job (2010),Though this movie is technically a documentary...
9,10. Boiler Room (2000),Boiler Room tells the story of Seth Davis (Gio...


In [3]:
# func to rearrange the order of columns.
def reorder_columns(df):
    new_order = ['movie', 'year', 'description', 'rating', 'minutes']
    return df[new_order]

# ddd the columns rating, minutes and year to the dataframe
df = (movies
 .assign(year=lambda df_: df_.title.str.extract(r'\((\d{4})\)'),
         movie=(lambda df_: df_.title.str[3:-7].str.lstrip()),
         rating=[7.1, 7.3, 7.8, 7.5, 8.2, 7.6, 6.6, 5.6, 8.2, 7, 6.4, 6.8, 6.2],
         minutes=(['1h 47m', '2h 6m', '2h 10m', '1h 58m', '3h', '1h 44m', '1h 47m', '1h 40m', '1h 48m', '2h', '1h 41m', '2h 13m', '1h 43m'])
        )
 .assign(minutes=lambda df_: df_.minutes.str.replace("h", '*60').str.replace(' ','+').str.replace('m','*1').apply(eval))
 .drop(columns='title')
 .pipe(reorder_columns)
)

__Getting ratings__  
I manually added the ratings for each movie. The ratings were obtained from IMDb.

### Create script dataframe
Here I'm creating another dataframe which contains the movie title based on the markdown file name and their corresponding script.

In [4]:
# load script dataset
transcript = pd.read_csv('f_csv/movie_2.csv')
transcript

,title,script
0,AmericanPsycho,"1<br>00:02:09,545 --> 00:02:12,757<br>rare roa..."
1,Arbitrage,"ï»¿1<br>00:00:55,387 --> 00:00:57,889<br><i>Bu..."
2,BoilerRoom,"1<br>00:00:01,000 --> 00:00:04,074<br>Subtitle..."
3,Equity,"ï»¿1<br>00:00:11,120 --> 00:00:13,112<br>[BELL..."
4,InsideJob,"ï»¿1<br>00:01:08,734 --> 00:01:10,634<br>Icela..."
5,MarginCall,"1<br>00:02:07,836 --> 00:02:09,338<br>Is that ..."
6,OtherPeoplesMoney,"1<br>00:00:16,150 --> 00:00:18,352<br>[Slinky ..."
7,RogueTrader,"1<br>00:00:14,000 --> 00:00:18,516<br>(newsrea..."
8,TheBigShort,"ï»¿1<br>00:00:19,900 --> 00:00:24,900<br><font..."
9,TheWizardOfLies,"ï»¿1<br>00:00:05,000 --> 00:00:10,000<br><font..."


## First round text cleaning
There are many unnecessary characters in the script text such as timestamps for each movie clip. I will tidy the script text in three phases.

In [5]:
def remove_numbers(dd, column_name):
    # Define a regular expression pattern that matches a number immediately followed by a letter
    pattern = re.compile(r'\d+([a-zA-Z])|\d\$|\d+\(')
    dd[column_name] = dd[column_name].str.replace(pattern, r'\1').str.strip()
    return dd

def replace_first_two_digits(dd, column_name):
    pattern = re.compile(r'\b(\d{2})(\d{4})\b')
    dd[column_name] = dd[column_name].str.replace(pattern, r'\2')
    return dd

def add_space_after_punctuation(dd, column_name):
    pattern = re.compile(r'([a-zA-Z])[?\.]([a-zA-Z])')
    dd[column_name] = dd[column_name].str.replace(pattern, r'\1? \2')
    return dd

def first_cleaning(dd):
    return (dd
     .assign(script=lambda dd_: dd_.script.str.replace('\d<br>\d{2}:\d{2}:\d{2},\d{3}\ -->\ \d{2}:\d{2}:\d{2},\d{3}<br>|ï»¿', '', regex=True),
             script1=lambda dd_: dd_.script.str.replace('(<br>)+\d|<br><br>|<br>|â™ªâ™ªâ™ª|\d-', ' ', regex=True),
             script2=lambda dd_: dd_.script1.str.replace('\d+\[\ \]|[â™ªâ™ªâ™ª]\ |</i>|<i>|â€¦|\ -\ ', '', regex=True),
             script3=lambda dd_: dd_.script2.str.replace('(<br>)+\d+|<br>-\ |<br>', ' ', regex=True),
             script4=lambda dd_: dd_.script3.str.replace('\[â™ªâ™ªâ™ª\]|(\d+)?\[((([A-Z])+ ?)+)+\]|\d+\*\*\ |\d+\.\.|\d+-\ ', '', regex=True),
             script5=lambda dd_: dd_.script4.str.replace('â€™', "'", regex=True),
             script6=lambda dd_: dd_.script5.str.replace(' [ ] ', '', regex=False),
             script7=lambda dd_: dd_.script6.str.replace('(\d+)?([A-Z])+:\ |<font\ color="#', '', regex=True),
            )
     .pipe(remove_numbers, 'script7')
     .pipe(replace_first_two_digits, 'script7')
     .pipe(add_space_after_punctuation, 'script7')
     .drop(columns=['script','script1','script2','script3','script4','script5','script6'])
     .rename(columns={'script7':'script'})
    )
    
dd = first_cleaning(transcript)
dd

,title,script
0,AmericanPsycho,rare roasted partridge breast in raspberry cou...
1,Arbitrage,But you took a huge bet on the housing crisis ...
2,BoilerRoom,Subtitles downloaded from www? OpenSubtitles? ...
3,Equity,MAN 1: The entire market is watching? WOMAN 1:...
4,InsideJob,Iceland is a stable democracy with a high stan...
5,MarginCall,Is that them? Jesus Christ. Are they going to ...
6,OtherPeoplesMoney,[Slinky clanking] I love money. I love money m...
7,RogueTrader,(newsreader) ..denied reports the British bank...
8,TheBigShort,"e020"">Cleaned, corrected and OCR issues fixed ..."
9,TheWizardOfLies,"e020"">HI removed, cleaned and corrected by Tro..."


## Second round text cleaning

In [6]:
def second_cleaning(df):
    return (dd
     .assign(script=lambda df_: df_.script.str.replace('([A-Z])+\ \d(\ )?:\ ', '', regex=True),
             script1=lambda df_: df_.script.str.replace('</font>', '', regex=False),
             script2=lambda df_: df_.script1.str.replace('e020">', '', regex=False),
             script3=lambda df_: df_.script2.str.replace('(â™)?(\d+)?â™', '', regex=True),
             script4=lambda df_: df_.script3.str.replace('\*(\ )?\*\ ', '', regex=True),
             script5=lambda df_: df_.script4.str.replace('(\d+)?(\*)?\ (\d+\*)?(\ )?', ' ', regex=True),
             script6=lambda df_: df_.script5.str.replace('(\d+)?\[([A-Za-z])+(\])?\ (([A-Za-z])+\ ([A-Za-z])+(\])?(\ )?([A-Za-z])+\]\ )?', '', regex=True),
             script7=lambda df_: df_.script6.str.replace('\d+\"', '"', regex=True),
             script8=lambda df_: df_.script7.str.replace('\([a-zA-Z]+((\ [a-zA-Z]+)+)?\)|\)|\.{3}|--|\d+\#\ ', '', regex=True),
             script9=lambda df_: df_.script8.str.replace('Subtitles downloaded from www? OpenSubtitles? org ', '', regex=False),
             script10=lambda df_: df_.script9.str.replace(' #', '', regex=False),
             script11=lambda df_: df_.script10.str.replace('Sync for "Wall? Street.1987.BluRay? P? DTS? x CHD" ::nlsinh@gmail? com:: WALLSTREET hddanang? com ', '', regex=False).str.strip()
            )
     .drop(columns=['script','script1','script2','script3','script4','script5','script6','script7','script8','script9','script10'])
     .rename(columns={'script11':'script'})
    )

dd = second_cleaning(dd)
dd

,title,script
0,AmericanPsycho,rare roasted partridge breast in raspberry cou...
1,Arbitrage,But you took a huge bet on the housing crisis ...
2,BoilerRoom,I read this article a while back that said Mic...
3,Equity,The entire market is watching? Back with Dynac...
4,InsideJob,Iceland is a stable democracy with a high stan...
5,MarginCall,Is that them? Jesus Christ. Are they going to ...
6,OtherPeoplesMoney,clanking] I love money. I love money more Than...
7,RogueTrader,..denied reports the British banking system is...
8,TheBigShort,"Cleaned, corrected and OCR issues fixed by Tro..."
9,TheWizardOfLies,"HI removed, cleaned and corrected by Tronar My..."


In [7]:
def third_cleaning(df):
    return (dd
     .assign(script=lambda df_: df_.script.str.replace('Best watched using Open Subtitles MKV Player', '', regex=False),
             script1=lambda df_: df_.script.str.replace('clanking]', '', regex=False),
             script2=lambda df_: df_.script1.str.replace(' Visiontext subtitles: Paul Sofer', '', regex=False),
             script3=lambda df_: df_.script2.str.replace('Cleaned, corrected and OCR issues fixed by Tronar Hiya,', '', regex=False),
             script4=lambda df_: df_.script3.str.replace('Sync for "Wall? Street.1987.BluRay? P.DTS? x CHD" ::nlsinh@gmail? com:: WALLSTREET hddanang? com ', '', regex=False).str.strip()
            )
     .drop(columns=['script','script1','script2','script3'])
     .rename(columns={'script4':'script'})
    )

dd = third_cleaning(dd)
dd

,title,script
0,AmericanPsycho,rare roasted partridge breast in raspberry cou...
1,Arbitrage,But you took a huge bet on the housing crisis ...
2,BoilerRoom,I read this article a while back that said Mic...
3,Equity,The entire market is watching? Back with Dynac...
4,InsideJob,Iceland is a stable democracy with a high stan...
5,MarginCall,Is that them? Jesus Christ. Are they going to ...
6,OtherPeoplesMoney,I love money. I love money more Than i love th...
7,RogueTrader,..denied reports the British banking system is...
8,TheBigShort,"Frank. How are the wife and kids? You know, fo..."
9,TheWizardOfLies,"HI removed, cleaned and corrected by Tronar My..."


### Make movies in both dataframes same
I want to have a single dataframe to work with, which means I’ll have to merge the two dataframes. Hence, I need to make sure that the names in the movie column match in both dataframes. This will allow me to merge the dataframes by joining on the movie column.

In [8]:
# give proper titles to the movie names in dd
mapping = {
        'AmericanPsycho':'American Psycho',
     'Arbitrage':'Arbitrage',
     'BoilerRoom':'Boiler Room',
     'Equity':'Equity',
     'InsideJob':'Inside Job',
     'MarginCall':'Margin Call',
     'OtherPeoplesMoney':'Other Peoples’ Money',
     'RogueTrader':'Rogue Trader',
     'TheBigShort':'The Big Short',
     'TheWizardOfLies':'The Wizard of Lies',
     'TheWolfofWallStreet':'The Wolf of Wall Street',
     'TradingPlaces':'Trading Places',
     'WallStreet':'Wall Street'
    }

dd = (dd
      .assign(title=dd.title.map(mapping))
     )

In [9]:
# Merge the dataframes on the movie and title columns.
df = df.merge(dd, left_on='movie', right_on='title', how='inner').drop(columns='title')
# df.to_csv('f_csv/finance_movies.csv', index=False)

# END